## Scraping Discogs.com with Beautiful Soup

Import packages

In [5]:
from bs4 import BeautifulSoup
import requests
import random
import pandas as pd
import re
import time
import numpy as np
import pickle
import os

Set the user agent to appear as though a browser is accessing the page

In [136]:
user_agent = {'User-agent': 'Mozilla/5.0'}

### Scraping the first few pages (search results of the top collected records) for over 1000 records and some data and links to pages that contain more data on each.

Set up the lists outside of the scraping loop, so if it fails some data is still saved

In [25]:
artists=[]
years=[]
release_countries=[]
labels=[]
artist_links=[]
albums=[]
album_links=[]

The loop below collects info from the first 20 pages of search results by appending numbers to the search result address

In [26]:
for i in range(1,21):
    '''collects most collected records data from discogs search pages'''
    #set the url based on the search page number
    if i==1:
        url = "https://www.discogs.com/search/?sort=have%2Cdesc&ev=em_rs&format_exact=Vinyl&layout=sm"
    if i > 0:
        url = "https://www.discogs.com/search/?sort=have%2Cdesc&ev=em_rs&format_exact=Vinyl&layout=sm&page=" + str(i)
    
    #wait to not get banned from discogs 
    time.sleep(10+2*random.random())
    
    #use response to get the page html
    response = requests.get(url, headers = user_agent)
    page = response.text
    
    #use beautiful soup to create a soup object of html to parse
    soup = BeautifulSoup(page, "lxml")
    
    #adding artists, artist links, albums, main album links, label, year, and country to the lists
    #if there are multiple artists or labels it takes only the first
    #try and except is used in case an element can't be found, numpy NaN is appended instead
    for element in soup.find(id='search_results').find_all(class_="card_body"):
        try:
            years.append(element.find_all(class_="card_release_year")[0].text)
        except:
            years.append(np.nan)
        try:
            labels.append(element.find('p', class_="card_info").find_all('a')[0].text)
        except:
            labels.append(np.nan)
        try:
            release_countries.append(element.find_all(class_="card_release_country")[0].text)
        except:
            release_countries.append(np.nan)
        try:
            albums.append(element.find_all("a", class_="search_result_title")[0].text)
        except:
            albums.append(np.nan)
        try:
            album_links.append('https://www.discogs.com'+element.find_all('a', class_="search_result_title")[0].get('href'))
        except:
            album_links.append(np.nan)
        try:
            artists.append(element.find_all('a')[0].text)
        except:
            artists.append(np.nan)
        try:
            artist_links.append('https://www.discogs.com'+element.find_all('a')[0].get('href'))
        except:
            artist_links.append(np.nan)

Note that when I originally did this, I had actually scraped the first page of search results twice, by starting at 0 rather than one. I corrected the mistake much later after noticing duplicates in my final data frame by using the below code:

In [ ]:
#top_1000_collected_final_df = top_1050_collected_final_df.drop(np.arange(50,100), axis=0)
#top_1000_collected_final_df.reset_index(inplace=True)

Check the length of all lists is equal and put into a pandas dataframe

In [34]:
top_all_collected = pd.DataFrame({'artists':artists,'albums':albums, 'artist_links':artist_links, 'album_links':album_links, 'release_year':years, 'label':labels, 'release_country':release_countries})

In [35]:
top_all_collected.tail()

,artists,albums,artist_links,album_links,release_year,label,release_country
1045,The Beatles,The Beatles At The Hollywood Bowl,https://www.discogs.com/artist/82730-The-Beatles,https://www.discogs.com/The-Beatles-The-Beatle...,1977,Parlophone,UK
1046,Fine Young Cannibals,The Raw & The Cooked,https://www.discogs.com/artist/4711-Fine-Young...,https://www.discogs.com/Fine-Young-Cannibals-T...,1988,London Records,UK
1047,Elton John,A Single Man,https://www.discogs.com/artist/57103-Elton-John,https://www.discogs.com/Elton-John-A-Single-Ma...,1978,The Rocket Record Company,UK
1048,David Bowie,Never Let Me Down,https://www.discogs.com/artist/10263-David-Bowie,https://www.discogs.com/David-Bowie-Never-Let-...,1987,EMI America,UK
1049,Daryl Hall + John Oates,H2O,https://www.discogs.com/artist/95886-Daryl-Hal...,https://www.discogs.com/Daryl-Hall-John-Oates-...,1982,RCA,US


In [58]:
top_all_collected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   artists          1050 non-null   object
 1   albums           1050 non-null   object
 2   artist_links     1050 non-null   object
 3   album_links      1050 non-null   object
 4   release_year     1047 non-null   object
 5   label            1050 non-null   object
 6   release_country  1049 non-null   object
dtypes: object(7)
memory usage: 57.5+ KB


In [38]:
with open('top_1050_collected_records.pickle', 'wb') as save_file:
    pickle.dump(top_all_collected, save_file)

## Scraping the main album pages for song info, and links to the first album version

Since albums come in many versions (or pressings), the price varies a lot based on the pressing number. Original pressings are worth the most usually. Since most popular albums were pressed around 100 times or more, I'd have to access that many pages to get the price for each pressing. I decided to focus on predicting the price of the first pressing.

In [42]:
from datetime import datetime

In [49]:
first_release_links = []
number_of_songs= []
average_song_length= []

The loop below scrapes the main album pages, gets the link to the first realse of that album, and also counts the number of songs, and calculates the average run time of a song on that album. 

In [50]:
for url in top_all_collected['album_links']:
    #from album main page
    time.sleep(10+2*random.random())
    response = requests.get(url, headers = user_agent)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #link to album 1st release page
    try:
        first_release_links.append('https://www.discogs.com' + soup.find('table', id='versions').find('td', class_='title').find_all('a')[0].get('href'))
    except:
        first_release_links.append(np.nan)
    
    #number of songs
    s=0
    try:
        for element in soup.find('table', class_="playlist").find_all('tr', class_="tracklist_track track"):
            for song in element.find('span', class_="tracklist_track_title"):
                s += 1
        number_of_songs.append(s)
    except:
        number_of_songs.append(np.nan)
    
    #average song length
    form='%M:%S'
    song_lengths=[]
    try:
        for element in soup.find('table', class_="playlist").find_all('tr', class_="tracklist_track track"):
            t = element.find('td', class_="tracklist_track_duration").find('span').text
            t = datetime.strptime(t,form)
            song_lengths.append(t)
        avg_time=datetime.strftime(datetime.fromtimestamp(sum(map(datetime.timestamp,song_lengths))/len(song_lengths)),"%M:%S")
        average_song_length.append(avg_time)
    except:
        average_song_length.append(np.nan)
        
        

In [51]:
album_main_pages = pd.DataFrame({'first_release_links':first_release_links,'number_of_songs':number_of_songs,'average_song_length':average_song_length})

I noticed that the number of songs was one lower than expected for each album. I am not sure why based on the code above, but will look into this. It's possible average song length is also missing one song. 

In [ ]:
album_main_pages['number_of_songs']=album_main_pages['number_of_songs']+1

In [56]:
album_main_pages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   first_release_links  1050 non-null   object
 1   number_of_songs      1050 non-null   int64 
 2   average_song_length  831 non-null    object
dtypes: int64(1), object(2)
memory usage: 24.7+ KB


In [54]:
with open('album_main_pages_2.pickle', 'wb') as save_file:
    pickle.dump(album_main_pages, save_file)

### Scraping the artist pages for data on number of albums, and years of albums

This uses the artist links from the top_1050_collected_records table. I thought that sorting the artist's albums in reverse to get the last year of a release would be a good idea, but a lot of the artists have compilation albums released years after the band broke up. Surprisingly, the year the band broke up is not readily available.

In [150]:
total_artist_albums = []
artist_first_years = []
artist_last_years = []

In [151]:
for url in top_1050_collected_records['artist_links']:
    #get info from artist pages
    response = requests.get(url, headers = user_agent)
    time.sleep(5+1*random.random())
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #total albums
    try:
        total_artist_albums.append(soup.find(id="discography_wrapper").find_all('span', class_='facet_count')[1].text)
    except:
        total_artist_albums.append(np.nan)
        
    #first album year
    try:
        artist_first_years.append(soup.find('table', id="artist").find_all('td', class_="year has_header")[0].text)
    except:
        artist_first_years.append(np.nan)
        
    #last album year
    try:
        url=url + "?sort=year%2Cdesc&limit=25"
        time.sleep(5+1*random.random())
        response = requests.get(url, headers = user_agent)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        artist_last_years.append(soup.find('table', id="artist").find_all('td', class_="year has_header")[0].text)
    except:
        artist_last_years.append(np.nan)
    

In [154]:
artist_data = pd.DataFrame({'total_artist_albums':total_artist_albums,'artist_last_years':artist_last_years,'artist_first_years':artist_first_years})

In [159]:
with open('artist_data.pickle', 'wb') as write_file:
    pickle.dump(artist_data, write_file)

### Scraping the first release page for each album for many stats and the target variable of median price

In [44]:
users_have = []
users_want = []
user_rating = []
lowest_price = []
median_price = []
highest_price = []
last_sold = []
number_for_sale = []
styles = []
genres = []
versions = []

In [57]:
for url in album_main_pages['first_release_links'][46:]:
    #get info from the first release of an album page
    response = requests.get(url, headers = user_agent)
    time.sleep(5+1*random.random())
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #getting info for the record stats, including price
    try:
        stats =[element for element in soup.find(id = "statistics").find_all('li')]
        
        users_have.append(re.search(':\n([0-9]*)',stats[0].text).group(1))
        users_want.append(re.search(':\n([0-9]*)',stats[1].text).group(1))
        user_rating.append(re.search(':\n([0-9].[0-9]{1,2})', stats[2].text).group(1))
        lowest_price.append(re.search('\$([0-9,]*.[0-9]{2})',stats[5].text).group(1))
        median_price.append(re.search('\$([0-9,]*.[0-9]{2})',stats[6].text).group(1))
        highest_price.append(re.search('\$([0-9,]*.[0-9]{2})',stats[7].text).group(1))
        last_sold.append(re.search(':\n([0-9]{2} [A-z]{3} [0-9]{2})',stats[4].text).group(1))
        
    except:
        users_have.append(np.nan)
        users_want.append(np.nan)
        user_rating.append(np.nan)
        lowest_price.append(np.nan)
        median_price.append(np.nan)
        highest_price.append(np.nan)
        last_sold.append(np.nan)

    #the number of versions
    try:
        versions.append(re.search('of ([0-9]*)\)',soup.find('h3',{'data-for':'.m_versions'}).text).group(1))
    except:
        versions.append(np.nan)
        
    #getting for sale info from marketplace section
    try:
        element = soup.find(class_ = "marketplace_for_sale_count").find('strong').text
        number_for_sale.append(re.search('([0-9,]*)',element).group())
    except:
        number_for_sale.append(np.nan)
        
    #find genre and styles (may be multiple, so a list)
    try:
        all_styles=[]
        for link in soup.find(class_="profile").find_all("a"):
            if 'style' in link.get('href'):
                all_styles.append(link.text)
        styles.append(all_styles)
    except:
        styles.append(np.nan)
        
    try:
        all_genres=[]
        for link in soup.find(class_="profile").find_all("a"):
            if 'genre' in link.get('href'):
                all_genres.append(link.text)
        genres.append(all_genres)
    except:
        genres.append(np.nan)
    

In [ ]:
first_version_page_df = pd.DataFrame({'users_have':users_have,'users_want':users_want,'user_rating':user_rating,'lowest_price':lowest_price,'median_price':median_price,'highest_price':highest_price,'last_sold':last_sold,'number_for_sale':number_for_sale,'styles':styles,'genres':genres,'versions':versions})

In [115]:
with open('first_edition_album_info.pickle', 'wb') as write_file:
    pickle.dump(first_edition_album_info, write_file)
    

In [112]:
first_edition_album_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            1050 non-null   int64 
 1   users_have       1032 non-null   object
 2   users_want       1032 non-null   object
 3   user_rating      1030 non-null   object
 4   lowest_price     1027 non-null   object
 5   median_price     1027 non-null   object
 6   highest_price    1027 non-null   object
 7   last_sold        1027 non-null   object
 8   styles           1049 non-null   object
 9   genres           1049 non-null   object
 10  number_for_sale  1034 non-null   object
 11  versions         1033 non-null   object
dtypes: int64(1), object(11)
memory usage: 98.6+ KB
